In [1]:
# Bibliotecas
import os
import zipfile
import tensorflow as tf
import numpy as np

In [2]:
# Definindo uma acurácia
acuracia_pret = 0.999

In [55]:
# Classe callback
class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>acuracia_pret):
            print('\n {0} accuracy reached'.format(acuracia_pret))
            self.model.stop_training=True
            
# Instanciando callback

callbacks = myCallBack()

In [13]:
# Download do arquivo
import wget
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip"
filename = wget.download(url)

100% [......................................................] 2670333 / 2670333

In [27]:
# Unzipando
zip_ref = zipfile.ZipFile('happy-or-sad.zip','r') # ler arquivo
zip_ref.extractall('h-or-s') # extraindo em
zip_ref.close() # fechando arquivo

In [173]:
# Criando modelo de Rede CNN
# Devido a complexidade do modelo iremos trabalhar com sucessivas combinações de Conv2D+Pooling antes de passar para
# a rede Multi Layer
model=tf.keras.models.Sequential([
    # Layers Convolucionais
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPool2D(2,2),
    # Flattening
    tf.keras.layers.Flatten(),
    # MLP
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [174]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 72, 72, 32)        18464     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9248)             

In [175]:
# Definindo função loss e optimizers

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [176]:
# Pegando os arquivos de imagem
# Importando biblioteca de criação do ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(rescale=1/255)
# Aqui podemos passar diversos parametros para alterar e criar novas imagens.
# Podemos criar imagens artificiais apenas rodando as imagens que temos
# Podemos usar funções de zoom, rescale, rotate...
# neste caso apenas instanciamos um datagenerator com rescale

train_generator = train_datagen.flow_from_directory(
    '/h-or-s', # pasta
    target_size=(150,150), # formato, vai aplicar em todas imagens
    batch_size=64, # lotes
    class_mode='binary' # tipo de classes
)

Found 80 images belonging to 2 classes.


In [179]:
# Fitando modelo
history = model.fit(
    train_generator,
    epochs=20,
    steps_per_epoch=40,
    callbacks=[callbacks]
)

# steps_per_epoch normalmente é numero de treino / batch_size
# Se tivessemos validation_steps seria numero de teste / batch_size

Epoch 1/20
40/40 [==============================] - 50s 1s/step - loss: 0.0286 - accuracy: 0.9912
Epoch 2/20
40/40 [==============================] - 51s 1s/step - loss: 0.1324 - accuracy: 0.9700
Epoch 3/20
40/40 [==============================] - ETA: 0s - loss: 1.8123e-04 - accuracy: 1.0000
 0.999 accuracy reached
40/40 [==============================] - 51s 1s/step - loss: 1.8123e-04 - accuracy: 1.0000
